In [21]:
from load_data import load

import numpy as np
from sklearn.model_selection import train_test_split

# Load the data if not loaded
if sim_arr is None:
    sim_arr, parameters_dict = load()


# Split the data into train, test and validation sets
train_set, test_set = train_test_split(sim_arr, test_size=0.2, random_state=42)
train_set, valid_set = train_test_split(
    train_set, test_size=0.2, random_state=42)

In [22]:
# check gpu
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [23]:
import optuna

from tensorflow.keras.layers import InputLayer, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import MeanSquaredError
from sklearn.model_selection import train_test_split

# Define input shape
input_shape = (80, 80, 3)
output_shape = (12, 1)


def create_model(trial):
    # Hyperparameters to optimize
    num_layers = trial.suggest_int('num_layers', 1, 5)
    filters = [trial.suggest_int(f'filters_{i}', 16, 128)
               for i in range(num_layers)]
    kernel_sizes = [trial.suggest_int(
        f'kernel_size_{i}', 2, 8) for i in range(num_layers)]
    strides = [trial.suggest_int(f'strides_{i}', 1, 2)
               for i in range(num_layers)]

    # Encoder
    encoder = Sequential(name = 'encoder')
    encoder.add(InputLayer(input_shape=input_shape))
    for i in range(num_layers):
        encoder.add(Conv2D(filters[i], kernel_sizes[i], strides=strides[i],
                           activation='selu', padding='same'))
        encoder.add(MaxPooling2D((2, 2), padding='same'))
   
    # Decoder
    decoder = Sequential(name = 'decoder')
    for i in range(num_layers):
        decoder.add(Conv2D(filters[num_layers - i - 1], kernel_sizes[num_layers -
                   i - 1], activation='selu', padding='same'))
        decoder.add(UpSampling2D((2, 2)))
    for i in range(num_layers):
        x = Conv2D(filters[i], kernel_sizes[i], strides=strides[i],
                   activation='selu', padding='same')(x)
        x = MaxPooling2D((2, 2), padding='same')(x)

    decoded = Conv2D(3, kernel_size=(
        3, 3), activation='selu', padding='same')(x)

    # Compile the model
    auto_encoder = Sequential([encoder, decoder], name="autoencoder")
    auto_encoder.compile(optimizer='adam', loss="MSE")

    return auto_encoder


def objective(trial):
    # Clear clutter from previous sessions
    tf.keras.backend.clear_session()

    # Create model
    model = create_model(trial)

    # Train the model
    model.fit(train_set, train_set, epochs=10, batch_size=32,
              validation_data=(valid_set, valid_set), verbose=0)

    # Evaluate the model
    score = model.evaluate(test_set, test_set, verbose=0)

    return score


# Perform hyperparameter optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

# Get the best hyperparameters
best_params = study.best_params
print("Best hyperparameters:", best_params)

[I 2024-03-15 05:20:59,600] A new study created in memory with name: no-name-eed627ed-8201-44f5-8b36-f9d5d75e439e
/tmp/ipykernel_32004/835762802.py:20: FutureWarning: suggest_int() got {'step', 'log'} as positional arguments but they were expected to be given as keyword arguments.
  kernel_sizes = [trial.suggest_int(
[W 2024-03-15 05:20:59,603] Trial 0 failed with parameters: {'num_layers': 4, 'filters_0': 78, 'filters_1': 98, 'filters_2': 95, 'filters_3': 20} because of the following error: ValueError('Samplers and other components in Optuna only accept step is 1 when `log` argument is True.').
Traceback (most recent call last):
  File "/home/tux/optim/.venv/lib/python3.11/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_32004/835762802.py", line 59, in objective
    model = create_model(trial)
            ^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_32004/835762802.py", line 20, 

ValueError: Samplers and other components in Optuna only accept step is 1 when `log` argument is True.

In [ ]:
# Train the model with the best hyperparameters
best_model = create_model(best_params)
history = best_model.fit(train_set, train_set, epochs=10, batch_size=32,
               validation_data=(valid_set, valid_set))